<a href="https://colab.research.google.com/github/padleh/Browser-Based-Tensorflow-js/blob/main/cloud_pacs_ai_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scrapping

## Import libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import deeplake

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model


In [4]:
pip install deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.

Scrapping the data

In [6]:
ds_train = deeplake.load('hub://activeloop/nih-chest-xray-train')
ds_test = deeplake.load('hub://activeloop/nih-chest-xray-test')

|

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/nih-chest-xray-train



/

...

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/nih-chest-xray-test



/

hub://activeloop/nih-chest-xray-test loaded successfully.



In [7]:
ds_train.visualize()

HINT: Please forward the port - 58675 to your local machine, if you are running on the cloud.


 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


In [8]:
ds_train.summary()

Dataset(path='hub://activeloop/nih-chest-xray-train', read_only=True, tensors=['findings', 'images', 'metadata/follow_up_num', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y', 'metadata/orig_img_w', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/patient_id', 'metadata/view_position'])

             tensor                  htype              shape             dtype  compression
             -------                -------            -------           -------  ------- 
            findings              class_label        (86524, 1:9)        uint32    None   
             images                  image     (86524, 1024, 1024, 1:4)   uint8     png   
     metadata/follow_up_num         generic           (86524, 1)          int32    None   
       metadata/orig_img_h          generic           (86524, 1)         uint32    None   
 metadata/orig_img_pix_spacing_x    generic           (86524, 1)         float32   None   
 metadata/orig_i

In [10]:
ds_test.summary()

Dataset(path='hub://activeloop/nih-chest-xray-test', read_only=True, tensors=['images', 'findings', 'boxes/bbox', 'boxes/finding', 'metadata/patient_id', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/follow_up_num', 'metadata/view_position', 'metadata/orig_img_w', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y'])

             tensor                  htype              shape             dtype  compression
             -------                -------            -------           -------  ------- 
             images                  image     (25596, 1024, 1024, 1:4)   uint8     png   
            findings              class_label        (25596, 1:8)        uint32    None   
           boxes/bbox                bbox          (25596, 0:4, 4)       float32   None   
          boxes/finding           class_label        (25596, 0:4)        uint32    None   
       metadata/patient_id          generic           (25596, 1)          int

In [14]:
img = ds_train.images[0].numpy()
img

array([[[202],
        [199],
        [195],
        ...,
        [  5],
        [  2],
        [  0]],

       [[199],
        [196],
        [195],
        ...,
        [  5],
        [  2],
        [  0]],

       [[196],
        [194],
        [193],
        ...,
        [  5],
        [  2],
        [  0]],

       ...,

       [[255],
        [255],
        [255],
        ...,
        [  0],
        [  0],
        [  0]],

       [[255],
        [255],
        [254],
        ...,
        [  0],
        [  0],
        [  0]],

       [[255],
        [255],
        [255],
        ...,
        [  0],
        [  0],
        [  0]]], dtype=uint8)

In [30]:
ds_train.findings[1].numpy()

array([6, 2], dtype=uint32)

In [76]:
# Indexing
img = ds_train.images[0].numpy()              # Fetch the 1st image and return a NumPy array
label = ds_train.findings[0].numpy(aslist=True) # Fetch the 1st label and store it as a
                                        # as a list

text_labels = ds_train.findings[1].data()['text'] # Fetch the first labels and return them as text

# Slicing
imgs = ds_train.images[0:100].numpy(aslist=True) # Fetch 100 images and return a NumPy array
                                # The method above produces an exception if
                                # the images are not all the same size

labels = ds_train.findings[0:100].numpy(aslist=True) # Fetch 100 labels and store
                                             # them as a list of NumPy arrays

/usr/local/lib/python3.10/dist-packages/deeplake/core/tensor.py:714: UserWarning: Indexing by integer in a for loop, like `for i in range(len(ds)): ... ds.tensor[i]` can be quite slow. Use `for i, sample in enumerate(ds)` instead.
  warnings.warn(


In [73]:
age = ds_train.metadata.patient_age[5].data()['value']
gender = ds_train.metadata.patient_gender[1].data()['text']
position = ds_train.metadata.view_position[1].data()['text']

In [78]:
print('finding is {}'.format(text_labels))
print('age is {}'.format(age))
print('gender is {}'.format(gender))
print('position is {}'.format(position))

finding is ['Cardiomegaly', 'Emphysema']
age is [69]
gender is ['M']
position is ['PA']
